<a href="https://colab.research.google.com/github/pankajtandon/Gist/blob/main/tell_me_a_story.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This will need to be done everytime your VM disconnects.

!pip install pyngrok -q
!pip install streamlit -q
!pip install openai -q
!pip install langchain -q
!pip install tiktoken -q
!pip install sentence_transformers -q
!pip install tiktoken -q
!pip install PyPDF2 -q
!pip install faiss-cpu -q
!pip install ipdb -q

In [5]:
import ipdb
import time
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from PIL import Image
import requests
from io import BytesIO

In [28]:
# This writes the code to the VM on which this notebook runs.

%%writefile /content/drive/MyDrive/colab_content/story.py



import ipdb
import time
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from PIL import Image
import requests
from io import BytesIO


PAGE_CONFIG = {"page_title": "Hello baby!", "page_icon": "smiley", "layout": "centered"}
st.set_page_config(**PAGE_CONFIG)
st.title("Welcome to our world of baby delights!")
st.subheader("We are head over heels!")


uploaded_file = st.file_uploader("Upload your image", type =  ['png', 'jpg'])

genre_choice = st.selectbox(
    label = 'Which genre should the story be in?',
    options= ['Scary', 'Funny', 'Horrific', 'Suspenseful', 'Poetic', 'Haiku Like', 'Pedantic', 'Shakesperean', 'Gothic', 'Romantic']
)

debug = st.checkbox("Would you like to see debug info?")

if (uploaded_file is not None) and (uploaded_file.name is not None):
  bytes = uploaded_file.getvalue()
  with open(uploaded_file.name, "wb") as file:
    file.write(bytes)

  st.image(uploaded_file, caption = "Uploaded Image", use_column_width = True)


image_to_text = pipeline("image-to-text", model = "Salesforce/blip-image-captioning-base")
# image_to_text = pipeline("image-to-text", model = "nlpconnect/vit-gpt2-image-captioning")

text = image_to_text(uploaded_file.name)
# text = image_to_text(img)

print(text[0]["generated_text"])


def story_teller(scenario, genre):
  template = """
  You are a great story teller! Based on the scenario below, tell me a story of about 25 words that is interesting and {genre}!
  Scenario: {scenario}
  """

  prompt = PromptTemplate(template = template, input_variables = ["scenario", "genre"])

  print("Prompt", prompt)

  story_teller_chain = LLMChain(llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 1), prompt = prompt, verbose = True)
  story = story_teller_chain.predict(scenario = scenario, genre = genre)
  return story

story = story_teller(scenario = text[0]["generated_text"], genre = genre_choice)

st.write("Here's your ", genre_choice, " story")
print(story)
st.write(story)



Overwriting /content/drive/MyDrive/colab_content/story.py


In [29]:
# Set up the tunnel to allow access to the running Streamlit instance.
import os
from pyngrok import ngrok

with open('/content/drive/MyDrive/colab_content/api-keys.txt', 'r') as f:
    api_key_list = f.readlines()
for kv in api_key_list:
    k,v = kv.split('=')
    #print(k, v)
    os.environ[k] = v.strip()

ngrok_token = os.getenv('NGROK_AUTH_TOKEN').strip()
!ngrok authtoken $ngrok_token
public_url = ngrok.connect(addr='8501') # This is the default Streamlit port
print('This is the URL that can be used to access the Streamlit app', public_url)



Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


This is the URL that can be used to access the Streamlit app NgrokTunnel: "https://9eca-34-80-55-159.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Start the streamlit app and leave it running and then access the running app at the URL above.

!streamlit run /content/drive/MyDrive/colab_content/story.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.55.159:8501

2023-06-21 02:47:03.271803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-21 02:47:12.336 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 552, in _run_script
    exec(code, module.__dict__)
  File "/content/drive/MyDrive/colab_content/story.py", line 49, in <module>
    text = image_to_text(uploaded_file.name)
AttributeError: 'NoneType' object has no attribute 'name'
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of 